In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-03-06 05:12:19.363077


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-03-06T05:00:00.000000000')

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [7]:
fetch_data_to

Timestamp('2025-03-06 05:00:00+0000', tz='UTC')

In [8]:
fetch_data_from

Timestamp('2025-02-05 05:00:00+0000', tz='UTC')

In [9]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

# def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
#     """
#     Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

#     Args:
#         from_date (datetime or str): The start date for the data batch.
#         to_date (datetime or str): The end date for the data batch.

#     Returns:
#         pd.DataFrame: A DataFrame containing the simulated production data.
#     """
#     # Convert string inputs to datetime if necessary
#     if isinstance(from_date, str):
#         from_date = datetime.fromisoformat(from_date)
#     if isinstance(to_date, str):
#         to_date = datetime.fromisoformat(to_date)

#     # Validate input dates
#     if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
#         raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
#     if from_date >= to_date:
#         raise ValueError("'from_date' must be earlier than 'to_date'.")

#     # Shift dates back by 52 weeks (1 year)
#     historical_from_date = from_date - timedelta(weeks=52)
#     historical_to_date = to_date - timedelta(weeks=52)

#     # Load and filter data for the historical period
#     rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
#     rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

#     if historical_to_date.month != historical_from_date.month:
#         rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
#         rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
#         # Combine the filtered data
#         rides = pd.concat([rides_from, rides_to], ignore_index=True)
#     else:
#         rides = rides_from
#     # Shift the data forward by 52 weeks to simulate recent data
#     rides['pickup_datetime'] += timedelta(weeks=52)

#     # Sort the data for consistency
#     rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

#     return rides

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks and remove timezone info
    historical_from_date = (from_date - timedelta(weeks=52)).replace(tzinfo=None)
    historical_to_date = (to_date - timedelta(weeks=52)).replace(tzinfo=None)

    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from['pickup_datetime'] = rides_from['pickup_datetime'].dt.tz_localize(None)
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to['pickup_datetime'] = rides_to['pickup_datetime'].dt.tz_localize(None)
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from

    rides['pickup_datetime'] += timedelta(weeks=52)
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [10]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [11]:
rides

,pickup_datetime,pickup_location_id
712157,2025-02-12 16:25:44,2
1018858,2025-02-15 16:56:40,2
442,2025-02-05 05:47:46,3
29233,2025-02-05 12:33:19,3
40516,2025-02-05 14:12:17,3
...,...,...
3026336,2025-03-06 04:09:27,263
2959137,2025-03-06 04:21:21,263
3026400,2025-03-06 04:31:14,263
3026340,2025-03-06 04:48:11,263


In [12]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [13]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-05 05:00:00,2,0
1,2025-02-05 06:00:00,2,0
2,2025-02-05 07:00:00,2,0
3,2025-02-05 08:00:00,2,0
4,2025-02-05 09:00:00,2,0
...,...,...,...
175387,2025-03-06 00:00:00,263,16
175388,2025-03-06 01:00:00,263,6
175389,2025-03-06 02:00:00,263,6
175390,2025-03-06 03:00:00,263,4


In [14]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175392 entries, 0 to 175391
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         175392 non-null  datetime64[ns]
 1   pickup_location_id  175392 non-null  int16         
 2   rides               175392 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [15]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature groups
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-03-06 00:12:35,713 INFO: Initializing external client
2025-03-06 00:12:35,727 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 00:12:38,480 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672


In [16]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |███████████████████████| Rows 175392/175392 | Elapsed Time: 00:35 | Remaining Time: 00:00
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/nyc_taxi25/Resources/jobs/time_series_hourly_feature_group_1_offline_fg_materialization/config_1741034887015) to trigger the materialization job again.


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)